In [ ]:
#import all libraries necessary for code
import pandas as pd
import datetime
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.auth.transport.requests import Request

# Introducing the bot and getting the user's name
def introduce_bot():
    print("Hello! I'm the REMAX chatbot!")
    name = input("What's your name? ")
    return name

#Main function to guide the user to our 4 possible scenarios
def main_interaction(name):
    """
    Main interaction function to guide the user through their options.
    """

    # Greet the user.
    print(f"\nHello {name}, what can I help you with today?")
    user_input = input().lower()

    # Define keyword groups for all 4 possible scenarios.
    buy_rent_keywords = ["buy", "rent"]
    sell_keywords = ["sell", "selling", "list", "listing"]
    agent_keywords = ["agent", "broker", "person", "someone"]
    open_house_keywords = ["open", "open house", "book", "viewing"]

    # Branching based on keywords in the user's input.
    if any(keyword in user_input for keyword in buy_rent_keywords):
        buy_rent_interaction()
    elif any(keyword in user_input for keyword in sell_keywords):
        sell_interaction()
    elif any(keyword in user_input for keyword in agent_keywords):
        agent_interaction()
    elif any(keyword in user_input for keyword in open_house_keywords):
        book_open_house()
    else:
        print("\nI'm sorry, I didn't understand that. Please try again.")

    # Collect feedback function for the end of the chatbot
    get_feedback()


# FIRST INTERACTION: Interaction for users interested in buying or renting property
def buy_rent_interaction():
    # Collecting user preferences in terms of type of property, the neighbourhood, budget, # of bed and # of baths
    buy_or_rent = input("Do you want to buy or rent a house? ").lower()
    property_type = input("What type of property are you looking for? (apartment, condo, house, townhouse): ").lower()
    neighborhood = input("Which neighbourhood are you interested in? (Plateau-Mont-Royal, Mile End, and so on): ").lower()
    budget = float(input("What's your budget? "))
    beds = int(input("How many bedrooms do you want? "))
    baths = int(input("How many bathrooms do you want? "))

    # Using the panda database to load and filter the housing data to match user preferences
    df = pd.read_excel('C:\\Users\\aadel\\OneDrive\\Documents\\Mcgill Masters\\1. Summer Term\\INSY 660 - Coding Foundations\\House_Info.xlsx')

    # Standardizing string columns for easier matching
    string_columns = ['Buy or rent', 'Type of property', 'Neighbourhood', 'Address']
    for col in string_columns:
        df[col] = df[col].str.lower().str.strip()

    # Converting price to float for easier calculations
    df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)

    # Applying filters based on user's input. Matches user input lines to columns in excel files.
    filtered_df = df[
        (df['Buy or rent'] == buy_or_rent) &
        (df['Type of property'] == property_type) &
        (df['Neighbourhood'] == neighborhood) &
        (df['Price'] <= budget) &
        (df['Beds'] == beds) &
        (df['Baths'] == baths)
    ]

    # Displaying results
    #Result to display if houses match the entered criteria:
    if not filtered_df.empty:
        print("\nBased on what you entered, here are a couple of houses that match what you're looking for:")
        for index, row in filtered_df.iterrows():
            print(row['Address'])
    #Result to display if no matches are found:
    else:
        print("\nSorry, no houses match your criteria.")


# SECOND SCENARIO: Interaction for users interested in selling
def sell_interaction():
    # Asking user if they have an agent. If not, prompting them to check our website.
    agent_response = input("Do you already have an agent? (yes/no) ").lower()
    if agent_response == "no":
        print("Please check our website for possible agents.")

    # Continuing to ask users for more information about the property they are selling (neighbourhood, property type, # beds, # baths)
    neighborhood = input("\nWhich neighbourhood is your house in? (Plateau-Mont-Royal, Mile End, and so on): ").lower()
    property_type = input("What type of property is it? (apartment, condo, house, townhouse): ").lower()
    beds = int(input("How many bedrooms does it have? "))
    baths = int(input("How many bathrooms does it have? "))

    # Using panda libraries to load and filter the excel file with the housing data to estimate the price
    df = pd.read_excel('C:\\Users\\aadel\\OneDrive\\Documents\\Mcgill Masters\\1. Summer Term\\INSY 660 - Coding Foundations\\House_Info.xlsx')

    # Standardizing string columns
    string_columns = ['Type of property', 'Neighbourhood', 'Address']
    for col in string_columns:
        df[col] = df[col].str.lower().str.strip()

    # Converting price to float
    df['Price'] = df['Price'].replace('[\$,]', '', regex=True).astype(float)

    # Filtering data to find similar properties. Matching the user input lines to the excel file columns.
    filtered_df = df[
        (df['Type of property'] == property_type) &
        (df['Neighbourhood'] == neighborhood) &
        (df['Beds'] == beds) &
        (df['Baths'] == baths)
    ]

    # Providing an estimated listing price based on similar properties
    if not filtered_df.empty:
  #Calculate the average price of all houses that match the criteria. Show the user the price average as input.
        avg_price = filtered_df['Price'].mean()
        print(f"\nBased on our internal databases of properties with the same characteristics, you should list your house for ${avg_price:,.2f}.")
  #If no houses are found, show a message to the user saying there are no houses that match the criteria.
    else:
        print("\nI apologize, we do not have information in our database for your property at the moment. Please try changing your inputs or try again at a later time.")


# THIRD SCENARIO: Interaction for users interested in finding an agent
def agent_interaction():

    # Gather user preferences for agent criteria
    neighborhood = input("\nWhich neighbourhood would you like the agent to be specialized in? ").lower()
    age_limit = int(input("Do you want the agent to be a minimum age? If so, specify the age. If not, enter 0: "))
    experience_limit = int(input("How many minimum years of experience do you want the agent to have? "))

    # Load agent data from the Excel spreadsheet.
    df_agents = pd.read_excel('C:\\Users\\aadel\\OneDrive\\Documents\\Mcgill Masters\\1. Summer Term\\INSY 660 - Coding Foundations\\Agent_Info.xlsx')

    # Ensure string columns are lowercase and stripped of whitespace
    string_columns = ['City', 'Country', 'Neighbourhood', 'Name', 'Phone', 'Email']  # Added 'Phone' and 'Email' to the list
    for col in string_columns:
        df_agents[col] = df_agents[col].str.lower().str.strip()

    # Filter the data based on user input
    filtered_df = df_agents[
        (df_agents['Neighbourhood'] == neighborhood) &
        (df_agents['Age'] >= age_limit) &
        (df_agents['Years of experience'] >= experience_limit)
    ]

        # Display results to the user.
    if not filtered_df.empty:
       #If agents are found, show the agents names and their contact information.
        print("\nHere are the agents and contact information that match your criteria:\n")
        for index, row in filtered_df.iterrows():
            # Capitalize the first letters of the agent's name
            name = row['Name'].title()

            # Include phone number and email address along with the agent's name
            print(f"{name}, Phone: {row['Phone']}, Email: {row['Email']}")
        print("""\nYou can either contact them directly, through our website,
or use this chatbot again to book an open house with them""")

#Show an apology message if no agents are found that meet the criteria.
    else:
        print("\nMy apologies, we do not have any agents that match what you are looking for.")


#FOURTH SCENARIO: Book an open house
#define a function to make sure the date is in the future.
def get_valid_date():

    #Prompt the user to enter a future date in a specific format.
    #Returns the date if valid.
    while True:
        date_str = input("\nEnter the date for the open house (dd-mm-yyyy): ")
        try:
    # Convert string input to a date object
            user_date = datetime.datetime.strptime(date_str, "%d-%m-%Y").date()
     # Ensure the provided date is in the future
            if user_date < datetime.datetime.now().date():
                print("Please enter a future date.")
                continue
            return user_date
        except ValueError:
            print("Invalid date format. Please use the format dd-mm-yyyy.")


def book_open_house():

# Allow the user to book an open house for a property with a specific agent
#Ask for the address and name of the agent the user wants to book the open house with.

    address = input("\nWhich property do you want to book the open house for? (Provide the address) ")
    agent_name = input("With which agent do you want to book it? ")

    # Fetch a valid date from the user. This uses the defined function get_valid_date
    desired_date = get_valid_date()

    # Get a valid time from the user, ensuring it's between 9am to 5pm.
    time = input("What time do you want to book it? (9am to 5pm, e.g., '2pm') ").lower()
    valid_times = [f"{i}am" for i in range(9, 12)] + [f"{i}pm" for i in range(1, 6)]
   #If the time is not between 9am and 5pm, ask the user to enter a new time.
    while time not in valid_times:
        print("Invalid time. Agents work from 9am to 5pm.")
        time = input("What time do you want to book it? (9am to 5pm, e.g., '2pm') ").lower()

    # Convert user time input to a time object. All open houses will be 2 hours, so add 2 hours to the start time to find the end time.
    desired_time = datetime.datetime.strptime(time, "%I%p").time()
    start_time = datetime.datetime.combine(desired_date, desired_time)
    end_time = start_time + datetime.timedelta(hours=2)

    # Schedule the open house in Google Calendar.
    book_in_google_calendar(agent_name, start_time, end_time)


def book_in_google_calendar(agent_name, start_time, end_time):
    # Schedule an open house in Google Calendar with the provided details.
    # Load Google Calendar API credentials.
    creds = None
    SCOPES = ['https://www.googleapis.com/auth/calendar']

    # Check for existing authentication token
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

    # Refresh or authenticate if necessary.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'C:\\Users\\aadel\\OneDrive\\Documents\\Mcgill Masters\\1. Summer Term\\INSY 660 - Coding Foundations\\client_secret_605178786772-f8i05dip8en2v1e3i26u11csavbkppcp.apps.googleusercontent.com.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    # Build the Google Calendar service object.
    service = build('calendar', 'v3', credentials=creds)

    # Convert start and end times to RFC3339 format
    event_start = start_time.isoformat()
    event_end = end_time.isoformat()

    # Create event details with America/Toronto timezone (Good for Montral). Name the event: Open house with (Agent's name).
    event = {
        'summary': f'Open House with {agent_name}',
        'start': {
            'dateTime': event_start,
            'timeZone': 'America/Toronto',
        },
        'end': {
            'dateTime': event_end,
            'timeZone': 'America/Toronto',
        },
    }

    # Add the event to Google Calendar
    event = service.events().insert(calendarId='primary', body=event).execute()
    print(f"\nOpen House booked with {agent_name} on {start_time.strftime('%A, %B %d, %Y')} at {start_time.strftime('%I:%M %p')}. Please check your calendar to see your booked time slot.")


# Get feedback function. At the end of each scenario, prompt the user to give feedback on the chatbot.

def get_feedback():
    """
    Gather feedback from the user about their experience with the chatbot.
    """
    feedback = input("\nDid you enjoy using this chatbot? (yes/no) ").lower()

#If user did not enjoy the chatbot, ask why not and return a message saying we will improve it based on their reason.
    if feedback == "no":
        reason = input("\nWe're sorry to hear that. Can you tell us why not? ")
        print(f"\nThank you for your feedback. We'll try to improve based on: {reason}")

#When user enters that they did enjoy the chatbot, thank them.
    else:
        print("\nThank you for the feedback! We're glad you enjoyed your experience.")


if __name__ == "__main__":
# Initial bot introduction and interaction sequence.
    user_name = introduce_bot()
    main_interaction(user_name)

